<a href="https://www.kaggle.com/code/emanarman/unique-solution-optimization?scriptVersionId=202613951" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#import libraries
import numpy as np
import pandas as pd

# As the forecasted power schedule is unavailable, the following lines create a simulation of it instead of uploading a dataset

# Create a date range for a single day with 15-minute intervals
time_index = pd.date_range(start='2024-10-11 00:00:00', end='2024-10-11 23:59:59', freq='15min')

# Generate random power data for the P_fc column
P_fc = np.random.uniform(low=80, high=120, size=len(time_index))  # Random values for P_fc

# Create a DataFrame with two columns: 'Time' and 'P_fc'
power_schedule = pd.DataFrame({
    'Time': time_index,
    'P_fc': P_fc,
})

# Print the first few rows of the schedule
print("Power Schedule:")
print(power_schedule.head())

# Initialize the output schedule
df = power_schedule.copy()
output_schedule = df.assign(H_1=0, H_2=0, P_E=0, Sur_elec=0)

#calculate the promised value for customer 1 at 90%  probability
# Step 1: Calculate the cumulative sum of the power output
power_schedule['Cumulative_Power'] = power_schedule['P_fc'].cumsum()

# Step 2: Calculate the 90th percentile of the cumulative power output
Q_prom = np.percentile(power_schedule['Cumulative_Power'], 90)


# Define the maximum capacities and demands
P_max = 300      # Maximum electrolyzer capacity (Given)
E_max = 200      # Maximum electricity can be sold to the grid (must be given too)
Q_lim_1 = 1.1*Q_prom    # Maximum hydrogen amount can be sold to customer 1 (Assume: Q_lim= 1.1*Q_prom)
Q_lim_2 = 0.5* Q_lim_1    # Maximum hydrogen amount can be sold to customer 2   (Assume: Q_lim_2 = 0.5 * Q_lim_1)
Q_max = Q_lim_1 + Q_lim_2  #maximum storage capacity, assume that maximum storage capacity equal to the maximum customer demand

print(Q_prom, Q_lim_1, Q_lim_2, Q_max)
# Initialize dummy variables to increment values in each time step
Q = Q_1 = Q_2 = E = 0


# Calculate the optimized values of electricity and hydrogen amount for each time step
for i, P_fc in enumerate(power_schedule['P_fc']):
    
    # Calculate the total optimal value of produced hydrogen
    H = min(P_max, P_fc, max(0, (Q_max -  Q))) # H= min{P_max, P_fc} & 0<= H<= Q_max
    # Cumulate the value to Q (total hydrogen production), assume as the storage quantity value
    Q += H
    
    # Calculate the optimal value of electricity to be sold to the grid
    P_E = output_schedule.loc[i, 'P_E'] = min((P_fc - H), max(0, (E_max -  E))) # P_E = P_fc - H  & 0 <= P_E <= E_max
    
    # Calculate the Surplus electricity if it exists
    output_schedule.loc[i, 'Sur_elec'] = max(0, P_fc - H - P_E)
    # Cumulate the value to E (total electricity sold)
    E += P_E
    
    # Calculate the optimal value of hydrogen amount sold to customer 1
    H_1 = output_schedule.loc[i, 'H_1'] =  min(H, max(0, (Q_lim_1 -  Q_1))) # H_1 = min(H, Q_lim_1) & 0<= H_1 <= Q_lim_1
    # Cumulate the value to Q_1 (total hydrogen to customer 1)
    Q_1 += H_1

    # Calculate the optimal value of hydrogen amount sold to customer 2
    H_2 = output_schedule.loc[i, 'H_2'] = min((H - H_1), max(0, (Q_lim_2 -  Q_2))) # H_2 = H - H_2  & 0<= H_2 <= Q_lim_2
    # Cumulate the value of Q_2 (total hydrogen to customer 2)
    Q_2 += H_2

# Print the final output schedule
print("\nFinal Output Schedule:")
print(output_schedule)

# Insufficiency Warning
if Q_1 < Q_prom:
    print('\nWarning: The hydrogen quantity is insufficient to meet the promised value to customer 1.')


Power Schedule:
                 Time        P_fc
0 2024-10-11 00:00:00   82.085959
1 2024-10-11 00:15:00  114.008589
2 2024-10-11 00:30:00  114.021161
3 2024-10-11 00:45:00  116.674098
4 2024-10-11 01:00:00  119.933927
8640.468670588933 9504.515537647827 4752.2577688239135 14256.77330647174

Final Output Schedule:
                  Time        P_fc         H_1        H_2  P_E  Sur_elec
0  2024-10-11 00:00:00   82.085959   82.085959   0.000000    0         0
1  2024-10-11 00:15:00  114.008589  114.008589   0.000000    0         0
2  2024-10-11 00:30:00  114.021161  114.021161   0.000000    0         0
3  2024-10-11 00:45:00  116.674098  116.674098   0.000000    0         0
4  2024-10-11 01:00:00  119.933927  119.933927   0.000000    0         0
..                 ...         ...         ...        ...  ...       ...
91 2024-10-11 22:45:00  110.543727  110.543727   0.000000    0         0
92 2024-10-11 23:00:00   82.757570   82.757570   0.000000    0         0
93 2024-10-11 23:15:00  11

/tmp/ipykernel_17/267795858.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '82.08595946201434' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  H_1 = output_schedule.loc[i, 'H_1'] =  min(H, max(0, (Q_lim_1 -  Q_1))) # H_1 = min(H, Q_lim_1) & 0<= H_1 <= Q_lim_1
/tmp/ipykernel_17/267795858.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '31.595102941552597' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  H_2 = output_schedule.loc[i, 'H_2'] = min((H - H_1), max(0, (Q_lim_2 -  Q_2))) # H_2 = H - H_2  & 0<= H_2 <= Q_lim_2
